<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/Lanngraph_corrective_rag_self_rag_and_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook we will try to create the production ready different RAG approaches that langgraph supports

### We will start with creating a corrective rag which will grade the document and generate the best answer